In [13]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [14]:
import torch

print('Pytorch version', torch.__version__)
if torch.backends.mps.is_available():
    print("mps")
elif torch.cuda.is_available():
    active_device = torch.device('cuda', 0)
else:
    active_device = torch.device('cpu')

Pytorch version 2.5.0.dev20240720
mps


In [15]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_data_df = pd.read_csv('../datasets/data_train.csv').head(1)
test_data_df = pd.read_csv('../datasets/data_test.csv').head(1)
valid_data_df = pd.read_csv('../datasets/data_valid.csv').head(1)

all_labels = pd.concat([train_data_df["label"], test_data_df["label"], valid_data_df["label"]])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

train_data_df["label"] = label_encoder.transform(train_data_df["label"])
test_data_df["label"] = label_encoder.transform(test_data_df["label"])
valid_data_df["label"] = label_encoder.transform(valid_data_df["label"])

In [16]:
from datasets import Dataset
train_data = Dataset.from_pandas(train_data_df)
test_data = Dataset.from_pandas(test_data_df)
valid_data = Dataset.from_pandas(valid_data_df)

In [17]:
train_data[0]

{'Unnamed: 0.1': 0,
 'Unnamed: 0': 15508,
 'patientid': 69318.0,
 'sex': None,
 'age': None,
 'ANA': None,
 'EXA': 'li. Knie: 0-5-130; kein Erguss; Lachman + unsicher; deutlihcer DS gesamter medialer GS; Varus + in 20°; Payr ++; McMurray und SteinmannI neg.; kein DS lateraler GS; Pivot shift nicht beurteilbar;',
 'DIA_text': 'Verstauchung und Zerrung des Kniegelenkes mit Riss des fibularen Seitenbandes [Außenband]',
 'DIA_code': 'S83.43 Z',
 'OP_text': 'Arthroskopische Operation am Gelenkknorpel und an den Menisken: Meniskusresektion; total',
 'OP_code': '5-812.6',
 'EXA_formatted': 'links knie keine überstreckung des knies streckdefizit des knies gute beugung des knies, kein schwellung, deutlihcer druckschmerz gesamter innereer seite des kniegelenks, kein druckschmerz äußereer seite des kniegelenks',
 'EXA_interpreted': 'Mein linkes Knie kann nicht überstreckt werden und ich habe ein Streckdefizit. Die Beugung des Knies ist gut möglich. Mein Knie hat keine Schwellung. Es gibt einen de

In [18]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-german-cased', device_map={"":"mps"})

def tokenize_function(examples):
    return tokenizer(examples["data"], padding="max_length", truncation=True, max_length=400)

In [19]:
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)
valid_dataset = valid_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [20]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=len(label_encoder.classes_), device_map={"":"mps"})

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy

# Defining the TrainingArguments() arguments
args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   eval_steps = 50, # Evaluation and Save happens every 50 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
   learning_rate=2e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=32,
   num_train_epochs=5,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True,
   use_mps_device = True)

/opt/homebrew/Caskroom/miniconda/base/envs/torch/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/torch/lib/python3.12/site-packages/transformers/training_args.py:2127: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [22]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [23]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [24]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

[codecarbon INFO @ 15:50:44] [setup] RAM Tracking...
[codecarbon INFO @ 15:50:44] [setup] GPU Tracking...
[codecarbon INFO @ 15:50:44] No GPU found.
[codecarbon INFO @ 15:50:44] [setup] CPU Tracking...
[codecarbon WARNING @ 15:50:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:50:44] We saw that you have a Apple M3 Max but we don't know it. Please contact us.
[codecarbon INFO @ 15:50:44] CPU Model on constant consumption mode: Apple M3 Max
[codecarbon INFO @ 15:50:44] >>> Tracker's metadata:
[codecarbon INFO @ 15:50:44]   Platform system: macOS-14.5-arm64-arm-64bit
[codecarbon INFO @ 15:50:44]   Python version: 3.12.3
[codecarbon INFO @ 15:50:44]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 15:50:44]   Available RAM : 64.000 GB
[codecarbon INFO @ 15:50:44]   CPU count: 16
[codecarbon INFO @ 15:50:44]   CPU model: Apple M3 Max
[codecarbon INFO @ 15:50:44]   GPU count: None
[codecarbon INFO @ 15:50:44]   GPU model: None


In [25]:
trainer.train()

  0%|          | 0/5 [00:00<?, ?it/s]

[codecarbon INFO @ 15:50:47] Energy consumed for RAM : 0.000300 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 15:50:47] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W


RuntimeError: Placeholder storage has not been allocated on MPS device!

[codecarbon INFO @ 15:50:47] 0.000832 kWh of electricity used since the beginning.


[codecarbon INFO @ 15:51:02] Energy consumed for RAM : 0.000100 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 15:51:02] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:51:02] 0.000277 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:51:02] Energy consumed for RAM : 0.000400 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 15:51:02] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:51:02] 0.001109 kWh of electricity used since the beginning.
